In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from math import *

%matplotlib inline

In [17]:
np.random.seed(12)
x = np.random.random(120)
y = np.random.random(120)-2
bidx = np.random.random(120)+1
askx = np.random.random(120)-0.5
bidy = np.random.random(120)+2
asky = np.random.random(120)-1.5

In [18]:
signal_exists = ['Y','N']         # exist, not exist
signal_opens  = ['O', 'C']        # open, close
signal_moves  = ['B', 'S']        # buy, sell

In [19]:
def generate_signal(priceA, priceB):
    '''Find the trading signal in one window, default as 60 days'''
    
    # test the existence of signal
    tfA = len(np.unique(np.isnan(priceA)))
    tfB = len(np.unique(np.isnan(priceB)))
            
    if  tfA == 2 or tfB == 2:
        signal_exist = signal_exists[0]
    else:
        signal_exist = signal_exists[1]

        # calculate return for two stocks
        Areturn = np.diff(priceA)/priceA[:(len(priceA)-1)]
        Breturn = np.diff(priceB)/priceB[:(len(priceB)-1)]

        # regression on return
        beta, beta0, r_value, p_value, std_err = stats.linregress(Areturn, Breturn)

        # get the residual epsilon_t
        e_t = np.array(Breturn - beta0 - beta*Areturn)

        # auxiliary process X_t
        Xt = []
        for i in range(len(Areturn)):
            Xt.append(np.sum(e_t[:i+1]))

        # regression on X_t
        length = len(Xt)
        Xt_vec = np.array(Xt)
        b, a, r_value_x, p_value_x, std_err_x = stats.linregress(Xt_vec[1:],Xt_vec[:length-1])

        # get the residual zeta_t
        z_t = Xt_vec[:length-1] - a - b*Xt_vec[1:]
        var_z = np.var(z_t)

        # calculate the s-score
        s_score = -a*sqrt(1-b**2)/((1-b)*sqrt(var_z)) + int(a/(1-b))*sqrt((1-b**2)/var_z)

        # trading signal
        if s_score < -1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[0]
        elif s_score > 1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[1]
        elif s_score < 0.75:
            signal_open = signal_opens[1]
            signal_move = signal_moves[0]
        elif s_score > -0.5:
            signal_open = signal_opens[1]
            signal_move = signal_moves[1]
        else:
            print('warning!')

        return beta,signal_exist, signal_open, signal_move

In [20]:
generate_signal(x,y)

(5.2076636313773747e-05, 'N', 'C', 'B')

In [21]:
position_dict = {}  # static variable: 'StockA': position
pnl = 0             # static variable

def build_position(position_dict, pnl, tickerA, tickerB, priceA, priceB, BidA, AskA, BidB, AskB, window):
    '''build position for each window
    
    Default parameters
    ------------------
    shares per trade:1000
    transaction cost fee: 0.0005
    '''
    transaction_cost = 0.0005
    
    # calculate signals for one window
    signals = []
    for i in range(window):
        current_priceA = priceA[i:i+window]
        current_priceB = priceB[i:i+window]
        result         = generate_signal(current_priceA, current_priceB)
        beta           = result[0]
        open_or_not    = result[2]
        buy_or_sell    = result[3]
        signals.append((beta, open_or_not, buy_or_sell))
    
    # for A
    pnlA = 0
    if tickerA in position_dict.keys():
        for i in range(window):
            if signals[i][1] == 'C':
                if signals[i][2] == 'B':
                    pnlA += (BidA[i+60]-transaction_cost)*min(position_dict[tickerA],1000*signals[i][0])
                    position_dict[tickerA] = max(position_dict[tickerA]-1000*signals[i][0],0)
                else:
                    pnlA -= (AskA[i+60]+transaction_cost)*min(position_dict[tickerA],1000*signals[i][0])
                    position_dict[tickerA] = max(position_dict[tickerA]-1000*signals[i][0],0)
            else:
                if signals[i][2] == 'B':
                    pnlA += (BidA[i+60]-transaction_cost)*1000*signals[i][0]
                    position_dict[tickerA] -= 1000*signals[i][0]
                else:
                    pnlA -= (AskA[i+60]+transaction_cost)*1000*signals[i][0]
                    position_dict[tickerA] += 1000*signals[i][0]
                    
    else:
        open_index = 0
        while signals[open_index][1] is not 'O':
            open_index += 1
            if open_index == (window-1):
                return position_dict, pnl
        if signals[open_index][2] == 'B':
            position_dict[tickerA] = -1000*signals[open_index][0]
            pnlA += -position_dict[tickerA]*(BidA[open_index+60]-transaction_cost)
            for i in range(open_index, window):
                if signals[i][1] == 'C':
                    pnlA -= (AskA[i+60]+transaction_cost)*min(position_dict[tickerA],1000*signals[i][0])
                    position_dict[tickerA] = max(position_dict[tickerA]-1000*signals[i][0],0)
        else:
            position_dict[tickerA] = 1000*signals[open_index][0]
            pnlA -= position_dict[tickerA]*(AskA[open_index+60]+transaction_cost)
            for i in range(open_index, window):
                if signals[i][1] == 'C':
                    pnlA += (BidA[i+60]-transaction_cost)*min(position_dict[tickerA],1000*signals[i][0])
                    position_dict[tickerA] = max(position_dict[tickerA]-1000*signals[i][0],0)    
    
    # for B
    pnlB = 0
    
    if tickerB in position_dict.keys():
        for i in range(window):
            if signals[i][1] == 'C':
                if signals[i][2] == 'B':
                    pnlB -= (AskB[i+60]+transaction_cost)*min(position_dict[tickerB],1000)
                    position_dict[tickerB] = max(position_dict[tickerB]-1000,0)
                else:
                    pnlB += (BidB[i+60]-transaction_cost)*min(position_dict[tickerB],1000)
                    position_dict[tickerB] = max(position_dict[tickerB]-1000,0)
            else:
                if signals[i][2] == 'B':
                    pnlB -= (AskB[i+60]+transaction_cost)*1000
                    position_dict[tickerB] += 1000
                else:
                    pnlB += (BidB[i+60]-transaction_cost)*1000
                    position_dict[tickerB] -= 1000
                    
    else:
        open_index = 0
        while signals[open_index][1] is not 'O':
            open_index += 1
            if open_index == (window-1):
                return position_dict, pnl
        if signals[open_index][2] == 'B':
            position_dict[tickerB] = 1000
            pnlB -= 1000*(AskB[open_index+60]+transaction_cost)
            for i in range(open_index, window):
                if signals[i][1] == 'C':
                    pnlB += (BidB[i+60]-transaction_cost)*min(position_dict[tickerB],1000)
                    position_dict[tickerB] = max(position_dict[tickerB]-1000,0)
        else:
            position_dict[tickerB] = -1000
            pnlB += 1000*BidB[open_index+60]
            for i in range(open_index, window):
                if signals[i][1] == 'C':
                    pnlB += -(AskB[i+60]+transaction_cost)*min(position_dict[tickerB],1000)
                    position_dict[tickerB] = max(position_dict[tickerB]-1000,0)
    pnl += pnlA+pnlB
    return position_dict, pnl

In [22]:
build_position(position_dict, pnl, 'A', 'B', x, y, bidx, askx, bidy,asky, 60)

({'A': 39.126783453912878, 'B': 0}, 4033.9464123420189)